So far we have prepared and visualized needed energy ranges. In order to validate these calculations and make an estimation of corresponding induced damage cross section, a series of TRIM calculations is needed. These calculations are done for a variety of ions and five energies between minimum and maximum for each. 
Following script is written in python and utilizes srim.py tools.
We are going to break script in smaller cells and run them in detail. 
Srimpy tools need to have been installed

1. LOAD REQUIRED LIBRARIES

In [5]:
import os
import shutil # for coping files 
from pprint import pprint
from oct2py import octave # for calling octave through python


import numpy as np
import matplotlib.pyplot as plt

from srim import TRIM, SR, Ion, Layer, Target
from srim.output import Results, Range


2. Extract required energy and ions' data from created files
    
    Files needed: "energy_ranges.dat", "elements_grep.txt"

In [6]:
# extract data from octave created .dat files containing energy information 
#  and "elements_grep.txt"
en_min = []
en_max = []
en_step = []
f = open('energy_ranges.dat','r') 
for line in f:
      line = line.strip()
      columns = line.split()
      energy_min = float(columns[1])
      energy_max = float(columns[2]) 
      ener_range = energy_max - energy_min
      steps = ener_range/4
      en_min.append(energy_min)
      en_max.append(energy_max)
      en_step.append(steps)


f.close()    

# %%
atom_numb = []
atom_name = []
g = open('elements_grep.txt','r')
for line in g:
    line2 = line.strip()
    columns2 = line.split()
    at_num = float(columns2[0])
    at_na = str(columns2[1])
    atom_numb.append(at_num)
    atom_name.append(at_na)

g.close()    
atom_numb = [int(x) for x in atom_numb]

3. Finally, the loop which executes TRIM for all energies, and ions asked follows.

Changes according to user's needs can be made. Lines that can usually be changed are:

ln 1: number of baem ions to be tested (ions are sorted with increasing atomic number)

ln 9-16: Target's properties 

ln 21: number_ions=*, calculation=1(Trim accepts no input from keeboard)

Finally, created and executable directories can be changed as needed

In [ ]:
for i in range(0,3):
  en = en_min[i]
  while en <= en_max[i]:
    print(en)
    ion = Ion(atom_name[i], energy=en*1.0e6)
    print(ion)
    # %%
## Make card for simulation [Energy] = eV , [width] = Angstr, [density] = gr/cm^3. 
    layer = Layer({
         'Fe' : {
              'stoich': 1.0,
              'E_d': 40.0,
              'lattice' : 0.0,
              'surface' : 4.34
            },
    }, density=7.865, width=40000.0)

    target = Target([layer])

# Initialize a TRIM calculation with given target and ion for 25 ions, quick calculation
    trim = TRIM(target, ion, number_ions=1000, calculation=1)

# Specify the directory of SRIM.exe
    srim_executable_directory = '/tmp/srim' 
    results = trim.run(srim_executable_directory)
  
# Transfer output files to a working directory of our choice
    name = str(en) + " MeV"
    parent_dir = "./" + atom_name[i] 
    path = os.path.join(parent_dir, name)
    os.makedirs(path, exist_ok=True)  # create the new outout file directory
    TRIM.copy_output_files('/tmp/srim', path ) # transfer output files to working directory
    
    en = en + en_step[i]

Output of all these runs are stored in the form of TRIM output data with the following data structure:

DATA_DIRECTORY/
- ION 1/
     -plot_cs.m
     -Energy1/Trim_Data+vac_txt_extract.m
     -Energy2/Trim_Data+vac_txt_extract.m
     -Energy3/Trim_Data+vac_txt_extract.m
     -Energy4/Trim_Data+vac_txt_extract.m
     -Energy5/Trim_Data+vac_txt_extract.m

- ION 2/
     -plot_cs.m
     -Energy1/Trim_Data+vac_txt_extract.m
     -Energy2/Trim_Data+vac_txt_extract.m
     -Energy3/Trim_Data+vac_txt_extract.m
     -Energy4/Trim_Data+vac_txt_extract.m
     -Energy5/Trim_Data+vac_txt_extract.m

          etc    

In [33]:
from oct2py import octave

for i in range(0,3):
  p_d1 = "./"+atom_name[i]
  name0 = atom_name[i]+"_vac_all.dat"
  path0 = os.path.join(p_d1,name0)
  try:
    os.remove(path0)
  except OSError:
    pass  
  en = en_min[i]
  while en <= en_max[i]:
    print(en)
    name = str(en) + " MeV"
    parent_dir = "./" + atom_name[i] 
    path = os.path.join(parent_dir, name)
    # copies octave script 
    fname = 'vac_txt_extract.m'
    dest = os.path.join(path, fname)
    #os.remove(dest)
    shutil.copyfile(fname, dest)
    h = open(dest, "a")
    h.write('info = [' + str(atom_numb[i]) + ' ' + str(en) + ' vac cs]')
    h.write('\n')
    h.write('save(\'../'+str(atom_name[i])+'_vac_all.dat\', \'info\', \'-append\', \'-ascii\')')
    h.close()
    # run octave script
    octave.run(dest)
    
    en = en + en_step[i]

  # create file "plot_cs.m" which plots damage cross section for each element
  name2 = 'plot_cs.m'
  path2 = os.path.join(parent_dir,name2)
  hh = open(path2, "w")
  hh.write('A = load(\''+str(atom_name[i])+'_vac_all.dat\')'+'\n')
  hh.write('E=A(:,2);'+'\n'+'cs = A(:,4)*1e19;'+'\n'+'\n'+'m1 = plot(E,cs,\'-o\')'+'\n'+'set (m1,\'linewidth\',2)'+'\n')
  hh.write('h=get(gcf, "currentaxes");'+'\n'+'set(h, "fontsize", 20, "linewidth", 2);'+'\n'+'grid'+'\n'+'\n')
  hh.write('for j=1:rows(cs),'+'\n'+'s{j} = [ \'\sigma = \' num2str(cs(j)) \'cm^2\'];'+'\n')
  hh.write('text(E(j),cs(j)-0.01,s{j},\'fontsize\', 8,\'fontweight\',\'bold\')'+'\n'+'end'+'\n'+'\n')
  hh.write('ylabel(\'Cross Section (x10^{-19} cm^2)\',\'fontsize\', 20)'+'\n')
  hh.write('xlabel(\'Ion energy (MeV)\',\'fontsize\', 20)'+'\n'+'title(\''+str(atom_name[i])+' Cross Section\')'+'\n'+'\n')
  hh.write('print -dpng '+atom_name[i]+'_damage_cs')
  hh.close()
  octave.run(path2)



0.8
info =

   1.0000e+00   8.0000e-01   8.8211e-03   1.0402e-19

error: 'sentinel' undefined near line 99 column 14
error: called from
    _pyeval at line 99 column 10
2.35
info =

   1.0000e+00   2.3500e+00   1.8614e-03   2.1951e-20

error: 'sentinel' undefined near line 99 column 14
error: called from
    _pyeval at line 99 column 10
3.9000000000000004
info =

   1.0000e+00   3.9000e+00   1.2769e-03   1.5058e-20

error: 'sentinel' undefined near line 99 column 14
error: called from
    _pyeval at line 99 column 10
5.45
info =

   1.0000e+00   5.4500e+00   1.2168e-03   1.4349e-20

error: 'sentinel' undefined near line 99 column 14
error: called from
    _pyeval at line 99 column 10
7.0
info =

   1.0000e+00   7.0000e+00   4.5530e-04   5.3691e-21

error: 'sentinel' undefined near line 99 column 14
error: called from
    _pyeval at line 99 column 10
A =

   1.0000e+00   8.0000e-01   8.8211e-03   1.0402e-19
   1.0000e+00   2.3500e+00   1.8614e-03   2.1951e-20
   1.0000e+00   3.9000e+00 